In [ ]:
!wget 'https://media.githubusercontent.com/media/2ai-lab/Machine-Learning-Tools/main/5. NLP/resources/datasets/Fake.csv?token=AFR3P6OHXIXM63KLALO3O7TEW4HGA' -O Fake.csv
!wget 'https://media.githubusercontent.com/media/2ai-lab/Machine-Learning-Tools/main/5.%20NLP/resources/datasets/True.csv?token=AFR3P6NBBO7VFQELI442YALEW4HPK' -O True.csv

--2023-07-19 01:15:40--  https://media.githubusercontent.com/media/2ai-lab/Machine-Learning-Tools/main/5.%20NLP/resources/datasets/Fake.csv?token=AFR3P6OHXIXM63KLALO3O7TEW4HGA
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-07-19 01:15:40 ERROR 404: Not Found.

--2023-07-19 01:15:40--  https://media.githubusercontent.com/media/2ai-lab/Machine-Learning-Tools/main/5.%20NLP/resources/datasets/True.csv?token=AFR3P6NBBO7VFQELI442YALEW4HPK
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-07-19 01:15:40 ERROR 404: 

In [ ]:
import torch
import pandas as pd
import nltk
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
# nlp library of Pytorch
from torchtext import data
import warnings as wrn
wrn.filterwarnings('ignore')

In [ ]:
true_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')
true_data.head()

EmptyDataError: ignored

In [1]:
true_data["label"] = np.ones(len(true_data),dtype=int)
fake_data["label"] = np.zeros(len(fake_data),dtype=int)

true_data.head()

NameError: ignored

In [ ]:
data = pd.concat((true_data,fake_data),axis=0)
print(data.info())

In [ ]:
data = data.sample(frac=1)

In [ ]:
data.to_csv("raw_data.csv", index = False)

In [ ]:
data=pd.read_csv("raw_data.csv")

In [ ]:
new_text = []
pattern = "[^a-zA-Z]"
lemma = nltk.WordNetLemmatizer()

for txt in data.text:

    txt = re.sub(pattern," ",txt) # Cleaning
    txt = txt.lower() # Lowering
    txt = nltk.word_tokenize(txt) # Tokenizing
    txt = [lemma.lemmatize(word) for word in txt] # Lemmatizing
    txt = " ".join(txt)
    new_text.append(txt)


In [ ]:
vectorizer_text = CountVectorizer(stop_words="english",max_features=4000)

In [ ]:
vectorized_text = vectorizer_text.fit_transform(new_text).toarray()

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, vectorized_text,labels):
        self.text = vectorized_text
        self.labels = labels

  def __len__(self):
        return len(self.labels)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        return self.text[index], self.labels[index]

In [ ]:
data['label'].values

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
ds = Dataset(vectorized_text, data['label'].values)

In [ ]:
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(ds, [0.7, 0.1, 0.2])

In [ ]:
from torch import nn
class LSTMNet(nn.Module):

    def __init__(self,
                 vocab_size,
                 embedding_dim,hidden_dim,
                 n_layers,bidirectional=True,
                 dropout=0.5)
    :

        super(LSTMNet,self).__init__()

        # Embedding layer converts integer sequences to vector sequences
        self.embedding = nn.Embedding(vocab_size,embedding_dim)

        # LSTM layer process the vector sequences
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout,
                            batch_first = True
                           )

        # Dense layer to predict
        self.fc = nn.Linear(hidden_dim * 2,1)
        # Prediction activation function
        self.sigmoid = nn.Sigmoid()


    def forward(self,text):
        embedded = self.embedding(text)


        output,(hidden_state,cell_state) = self.lstm(embedded)

        # Concatenating the final forward and backward hidden states
        hidden = torch.cat(
            (hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1
            )

        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.sigmoid(dense_outputs)

        return outputs

In [ ]:
train_dataset.__getitem__(1)

(array([0, 0, 0, ..., 0, 0, 0]), 0)

In [ ]:
batch_size = 32
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [1]:
from torch import optim
model = LSTMNet(4000, 1000, 64, 2)
model.to('cuda')

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10


NameError: ignored

In [ ]:
for epoch in range(num_epochs):
    #Training
    model.train()
    print("EPOCH:", epoch)
    for i, (inputs, labels) in enumerate(train_data_loader):
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        outputs = outputs.to('cuda')
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

    #validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (inputs, labels) in enumerate(val_data_loader):
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')
          outputs = model(inputs)
          predicted = (outputs >= 0.5).squeeze().long()
          total += labels.size(0)
          correct += (predicted == labels.squeeze()).sum().item()  # Squeeze labels for comparison
          accuracy = correct / total
        print('Test Accuracy: {:.2f}%'.format(accuracy * 100))



EPOCH: 0


NameError: ignored

In [ ]:
steps = int(len(train_data_loader)/982 )

In [ ]:
steps

1